In [16]:
# %matplotlib inline
import pandas as pd
import plotly.express as px

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import widgets
from ipywidgets import interact, interact_manual, Layout
from IPython.display import display
from IPython.display import display, clear_output

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sp
import matplotlib.dates as mdates


from jupyter_dash import JupyterDash
from dash import Dash, html, dcc


# Getting data in from CSVs to dataframes
ChildIdentifiers = pd.read_csv('/workspaces/LA-Analyst-Tinkering/FAKE-CIN-DATA/FakeChildIdentifiers.csv')
CINdetails = pd.read_csv('/workspaces/LA-Analyst-Tinkering/FAKE-CIN-DATA/FakeCINdetails.csv')
ChildCharacteristics = pd.read_csv('/workspaces/LA-Analyst-Tinkering/FAKE-CIN-DATA/FakeChildCharacteristics.csv')
ethnicities_sub = ChildCharacteristics['Ethnicity'].value_counts()

groupings = {'White' : ['WOTH', 'WIRI', 'WROM', 'WIRT', 'WBRI',],
            'Black' : ['BCRB', 'BAFR', 'BOTH', ],
            'Asian' : ['APKN', 'ABAN', 'AIND', 'CHNE', 'AOTH'],
            'Mixed' : ['MWAS', 'MWBA', 'MOTH', 'MWBC',],
            'Other' : ['REFU', 'NOBT', 'OOTH'],}

# It would be possible to do this with the pandas .map() fucntion if the dicitonary was flat, as in 1 key to 1 vlaue, not a dict of lists.
for k, v in groupings.items():
    ChildCharacteristics.loc[ChildCharacteristics['Ethnicity'].isin(groupings[k]), 'macro_groups'] = k

ethnicities_main = ChildCharacteristics['macro_groups'].value_counts()
group_types = {'Main group' : ethnicities_main, 'Subgroup' : ethnicities_sub}

ethnicity_groupings = widgets.Dropdown(
    options=['Main group', 'Subgroup'],
    value='Main group',
    description='Ethnicity main or subgroups',
)

    
def sample_graph(years, line_on, ethnicities):
    ChildIdentifiers = pd.read_csv('/workspaces/LA-Analyst-Tinkering/FAKE-CIN-DATA/FakeChildIdentifiers.csv')
    CINdetails = pd.read_csv('/workspaces/LA-Analyst-Tinkering/FAKE-CIN-DATA/FakeCINdetails.csv')

    merged_df = CINdetails.merge(ChildIdentifiers, left_on='LAchildID', right_on='LAchildID', how='left').merge(ChildCharacteristics, left_on='LAchildID', right_on='LAchildID', how='left')


    #df = merged_df[['LAchildID', 'PersonBirthDate', 'CINreferralDate', 'CINclosureDate', 'macro_groups', 'Ethnicity']]
    # df_dates['PersonBirthDate'] = pd.to_datetime(df['PersonBirthDate'], format='%Y/%m/%d')
    # df_dates['CINreferralDate'] = pd.to_datetime(df['CINreferralDate'], format='%Y/%m/%d')
    # df_dates['CINclosureDate'] = pd.to_datetime(df['CINclosureDate'], format='%Y/%m/%d')
    df_dates = merged_df.loc[:,['PersonBirthDate', 'CINreferralDate', 'CINclosureDate']].apply(
       pd.to_datetime, format='%Y/%m/%d', errors='coerce'
    )
    df_dates['macro_groups'] = merged_df['macro_groups']
    df_dates = df_dates[df_dates['macro_groups'].isin(ethnicities)]
    

    df_dates = df_dates[df_dates['CINclosureDate'].notna()]

    df_dates['CINrefToClose'] = (df_dates['CINclosureDate'] - df_dates['CINreferralDate'])
    df_dates['CINrefToClose'] = df_dates['CINrefToClose'].astype('timedelta64[h]')
    df_dates['CINrefToClose'] = df_dates['CINrefToClose'].astype(float) / 24
    
    df_dates = df_dates[(df_dates['PersonBirthDate'].dt.year >= years[0]) & (df_dates['PersonBirthDate'].dt.year <= years[1])]

    
    if line_on == 'Regression on':
        fig = px.scatter(df_dates, x='PersonBirthDate', y='CINrefToClose', trendline='ols', trendline_scope='overall',
            labels = {'PersonBirthDate' : 'Birth date',
                      'CINrefToClose': 'Wait between first referral and cin closure(days)',
                      'macro_groups': 'Ethnicity'},
            title = ('Time between first referral and closure date by birth date'),
            color = 'macro_groups')
    else:
        fig = px.scatter(df_dates, x='PersonBirthDate', y='CINrefToClose',
            labels = {'PersonBirthDate' : 'Birth date',
                      'CINrefToClose': 'Wait between first referral and cin closure(days)',
                      'macro_groups': 'Ethnicity'},
            title = ('Time between first referral and closure date by birth date'),
            color = 'macro_groups')
    fig.show()
   
    

year_select = widgets.IntRangeSlider(
    value = [2004, 2022],
    min = 2004,
    max = 2022,
    step = 1,
    description = 'Year Range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout = True,
    readout_format = 'd',
    layout={'width': '500px'}
    )

line_select = widgets.RadioButtons(
    options=['Regression on', 'Regression off',],
#     value='pineapple',
    description='Regression line',
    disabled=False
)

ethnic_select = widgets.SelectMultiple(
    options=['White', 'Black', 'Asian', 'Mixed', 'Other'],
    value=['White', 'Black', 'Asian', 'Mixed', 'Other'],
    #rows=10,
    description='Ethnicties to view',
    disabled=False
)

interact(sample_graph, years=year_select, line_on=line_select, ethnicities = ethnic_select)




interactive(children=(IntRangeSlider(value=(2004, 2022), continuous_update=False, description='Year Range:', l…

<function __main__.sample_graph(years, line_on, ethnicities)>